In [142]:
import Noise_Interferometry
import pickle
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import math as m

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [143]:
years = [2015, 2016, 2017, 2018, 2019, 2020]
peak_names = ['dA','s1b0A','s2b1A','dB','s1b0B','s2b1B']
avg_time = 601

SNR_by_year = []
peak_idx_by_year = []
dates = []
num_availables = []

for year in years:
    print(f'Loading {year}...')
    
    file_name = f'/Volumes/Ocean_Acoustics/NCCFs/MJ03F-MJ03E/Longterm_MA_{year}_{avg_time}.pkl'
    long_term = pickle.load(open(file_name,'rb'))
    dates.append(long_term.dates)
    num_availables.append(np.array(long_term.num_available))
    
    SNR_by_peak = []
    peak_idx_by_peak = []
    for peak_name in peak_names:
        print(f'   Calculating {peak_name}...')
        SNR, peak_idx, _ = long_term.snr_of_peak_amp(peak_name)
        SNR_by_peak.append(SNR)
        peak_idx_by_peak.append(peak_idx)
        
    SNR_by_year.append(SNR_by_peak)
    peak_idx_by_year.append(peak_idx_by_peak)

Loading 2015...
   Calculating dA...
   Calculating s1b0A...
   Calculating s2b1A...
   Calculating dB...


/Users/jhrag/Code/ocean_acoustics/Noise_Interferometry/Modules/analysis.py:1200: UserWarning: Peak index jumps more than 3 in single step
  warnings.warn('Peak index jumps more than 3 in single step')


   Calculating s1b0B...
   Calculating s2b1B...
Loading 2016...
   Calculating dA...
   Calculating s1b0A...
   Calculating s2b1A...
   Calculating dB...
   Calculating s1b0B...
   Calculating s2b1B...
Loading 2017...
   Calculating dA...
   Calculating s1b0A...
   Calculating s2b1A...
   Calculating dB...
   Calculating s1b0B...
   Calculating s2b1B...
Loading 2018...
   Calculating dA...
   Calculating s1b0A...
   Calculating s2b1A...
   Calculating dB...
   Calculating s1b0B...
   Calculating s2b1B...
Loading 2019...
   Calculating dA...
   Calculating s1b0A...
   Calculating s2b1A...
   Calculating dB...
   Calculating s1b0B...
   Calculating s2b1B...
Loading 2020...
   Calculating dA...
   Calculating s1b0A...
   Calculating s2b1A...
   Calculating dB...
   Calculating s1b0B...
   Calculating s2b1B...


## Create Num_hours Threshold and create masked array

In [144]:
threshold = 100

SNRs_mask = []

for year in range(6):
    SNR_peak_mask = []
    for peak in range(6):
        mask = (~(num_availables[year] >= threshold))
        SNR_peak_mask.append(np.ma.masked_array(SNR_by_year[year][peak],mask=mask))
    SNRs_mask.append(SNR_peak_mask)


# Create Yearlong SNR_Plots

In [145]:
peak_names = ['dA','s1b0A','s2b1A','dB','s1b0B','s2b1B']
peak_count = 0
for peak_name in peak_names:  
    print(f'Plotting for {peak_name}')
    fig, axes = plt.subplots(2,3,figsize=(15,5))
    k = 0
    for ax in fig.get_axes():
        ax.plot(dates[k], SNRs_mask[k][peak_count])

        plt.sca(ax)
        plt.grid()
        plt.title(f'SNR for {years[k]}')
        plt.xticks(rotation=20)
        plt.ylabel('SNR [dB]')
        k += 1

    plt.tight_layout()

    fig.savefig(f'longterm_SNR_figures/{avg_time}/SNRs_by_year_{peak_name}.png',dpi=300)
    plt.close()
    
    peak_count += 1

Plotting for dA
Plotting for s1b0A
Plotting for s2b1A
Plotting for dB
Plotting for s1b0B
Plotting for s2b1B


In [146]:
fig, axes = plt.subplots(2,3,figsize=(15,5))
k = 0
for ax in fig.get_axes():
    ax.plot(dates[k], num_availables[k])

    plt.sca(ax)
    plt.grid()
    plt.title(f'Num Available for {years[k]}')
    plt.xticks(rotation=20)
    plt.ylabel('SNR [dB]')
    k += 1

plt.tight_layout()

fig.savefig(f'longterm_SNR_figures/{avg_time}/num_available_by_year.png',dpi=300)
plt.close()


In [147]:
peak_names = ['dA','s1b0A','s2b1A','dB','s1b0B','s2b1B']
peak_count = 0
for peak_name in peak_names:  
    print(f'Plotting for {peak_name}')
    fig, axes = plt.subplots(2,3,figsize=(15,5))
    k = 0
    for ax in fig.get_axes():
        ax.plot(dates[k], peak_idx_by_year[k][peak_count])

        plt.sca(ax)
        plt.grid()
        plt.title(f'Peak Index for {years[k]}')
        plt.xticks(rotation=20)
        plt.ylabel('Peak Index [time bin]')
        k += 1

    plt.tight_layout()

    fig.savefig(f'longterm_SNR_figures/{avg_time}/Peak_Idx_by_year_{peak_name}.png',dpi=300)
    plt.close()
    
    peak_count += 1

Plotting for dA
Plotting for s1b0A
Plotting for s2b1A
Plotting for dB
Plotting for s1b0B
Plotting for s2b1B


# Month Long Bar Plots

In [148]:
for peak in range(6):
    # Generate Dataframe with [year] [month] [snr] columns

    SNR_month_all = []
    for k in range(6):
        num_hours = len(SNRs_mask[k][peak])
        num_split = m.trunc(num_hours/12)*12

        x = np.split(SNRs_mask[k][peak][:num_split],12)

        SNR_month = np.zeros((12,1))
        for n in range(12):
            SNR_month[n] = np.mean(x[n])

        SNR_month_all.append(SNR_month)

    SNR_month_all_array = np.ndarray.flatten(np.array(SNR_month_all))
    years = [2015, 2016, 2017, 2018, 2019, 2020]
    years2 = [val for val in years for _ in range(12)]
    months = ['January','February','March','April','May','June','July','August','September','October','November','December']*6

    d = {'year':years2, 'month':months, 'SNR':SNR_month_all_array}

    df = pd.DataFrame.from_dict(data=d)
    
    cp = sns.color_palette('Set2')
    fig1, ax1 = plt.subplots(1,1,figsize=(10,7))

    ax1 = sns.barplot(x="month", y="SNR", hue="year", data=df, palette=cp)
    plt.sca(ax1)
    plt.xticks(rotation=45)
    plt.legend(loc="lower right", ncol=len(df.columns))
    plt.title('Average SNR for Each Month')
    plt.ylabel('SNR [dB]')

    fig1.savefig(f'longterm_SNR_figures/{avg_time}/SNR_for_every_month_{peak_names[peak]}.png',dpi=200)
    
    plt.close()
    
    cp = sns.color_palette('Set2')
    fig2, ax2 = plt.subplots(1,1,figsize=(10,7))

    ax2 = sns.barplot(x="month", y="SNR", data=df, palette=cp)
    plt.sca(ax2)
    plt.xticks(rotation=45)
    plt.title('Average SNR for Each Month')
    plt.ylabel('SNR [dB]')

    fig2.savefig(f'longterm_SNR_figures/{avg_time}/SNR_for_every_month_avg_{peak_names[peak]}.png',dpi=200)
    plt.close()

In [149]:
peak_names[2]

's2b1A'